In [2]:
import tkinter as tk
# import all the required modules
import socket
import threading
from tkinter import *
from tkinter import font
from tkinter import ttk
import time

PORT = 1234
SERVER = "172.17.0.152"
ADDRESS = (SERVER, PORT)
FORMAT = "utf-8"
# Create a new client socket and connect to the server
client = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
client.connect(ADDRESS)

def start_chat():
    
    # GUI class for the chat
    class GUI:
        # constructor method
        
        def __init__(self):
            # chat window which is currently hidden
            self.Window = Tk()
            self.Window.withdraw()
            
            # login window
            self.login = Toplevel()
            # set the title
            self.login.title("Login")
            self.login.resizable(width=False,
                                height=False)
            self.login.configure(width=400,
                                height=300)
            # create a Label
            self.pls = Label(self.login,
                            text="Please login to continue",
                            justify=CENTER,
                            font="Helvetica 14 bold")

            self.pls.place(relheight=0.15,
                        relx=0.2,
                        rely=0.07)
            # create a Label
            self.labelName = Label(self.login,text="Name: ",
                                font="Helvetica 12")

            self.labelName.place(relheight=0.2,
                                relx=0.1,
                                rely=0.2)

            # create a entry box for
            # tyoing the message
            self.entryName = Entry(self.login,font="Helvetica 14")

            self.entryName.place(relwidth=0.4,
                                relheight=0.12,
                                relx=0.35,
                                rely=0.2)

            # set the focus of the cursor
            self.entryName.focus()

            # create a Continue Button
            # along with action
            self.go = Button(self.login,text="CONTINUE",font="Helvetica 14 bold",command=lambda: self.goAhead(self.entryName.get()))

            self.go.place(relx=0.4,rely=0.55)
            self.Window.mainloop()
        
        def goAhead(self, name):
            self.login.destroy()
            self.layout(name)
            # the thread to receive messages
            rcv = threading.Thread(target=self.receive)
            rcv.start()

        # The main layout of the chat
        def layout(self, name):
            self.name = name
            # to show chat window
            client.send(self.name.encode(FORMAT))
            self.Window.deiconify()
            self.Window.title("CHATROOM")
            self.Window.resizable(width=False,height=False)
            self.Window.configure(width=470,height=550,bg="#17202A")
            self.labelHead = Label(self.Window,
                                bg="#17202A",
                                fg="#EAECEE",
                                text=self.name,
                                font="Helvetica 13 bold",
                                pady=5)

            self.labelHead.place(relwidth=1)
            self.line = Label(self.Window,width=450,bg="#ABB2B9")

            self.line.place(relwidth=1,
                            rely=0.07,
                            relheight=0.012)

            self.textCons = Text(self.Window,
                                width=20,
                                height=2,
                                bg="#17202A",
                                fg="#EAECEE",
                                font="Helvetica 14",
                                padx=5,
                                pady=5)

            self.textCons.place(relheight=0.745,
                                relwidth=1,
                                rely=0.08)

            self.labelBottom = Label(self.Window,
                                    bg="#ABB2B9",
                                    height=80)

            self.labelBottom.place(relwidth=1,
                                rely=0.825)

            self.entryMsg = Entry(self.labelBottom,
                                bg="#2C3E50",
                                fg="#EAECEE",
                                font="Helvetica 13")

            # place the given widget into the gui window
            self.entryMsg.place(relwidth=0.74,
                                relheight=0.06,
                                rely=0.008,
                                relx=0.011)

            self.entryMsg.focus()

            # create a Send Button
            self.buttonMsg = Button(self.labelBottom,
                                    text="Send",
                                    font="Helvetica 10 bold",
                                    width=20,
                                    bg="#ABB2B9",
                                    command=lambda: self.sendButton(self.entryMsg.get()))

            self.buttonMsg.place(relx=0.57,
                                rely=0.008,
                                relheight=0.06,
                                relwidth=0.22)
            # create a Send Button
            self.buttonMsg = Button(self.labelBottom,
                                    text="exit",
                                    font="Helvetica 10 bold",
                                    width=20,
                                    bg="#ABB2B9",
                                    command=lambda: self.exitButton())

            self.buttonMsg.place(relx=0.77,
                                rely=0.008,
                                relheight=0.06,
                                relwidth=0.22)

            self.textCons.config(cursor="arrow")

            # create a scroll bar
            scrollbar = Scrollbar(self.textCons)

            # place the scroll bar into the gui window
            scrollbar.place(relheight=1,relx=0.974)

            scrollbar.config(command=self.textCons.yview)

            self.textCons.config(state=DISABLED)

        # function to basically start the thread for sending messages
        def sendButton(self, msg):
            self.textCons.config(state=DISABLED)
            self.msg = msg
            self.entryMsg.delete(0, END)
            snd = threading.Thread(target=self.sendMessage)
            snd.start()
        
        def exitButton(self):
            
            self.Window.destroy()
        
        # function to receive messages
        def receive(self):
            while True:
                try:
                    message = client.recv(1024).decode(FORMAT)
                    # if the messages from the server is NAME send the client's name
                    if message == 'NAME':
                        client.send(self.name.encode(FORMAT))
                    else:
                        # insert messages to text box
                        self.textCons.config(state=NORMAL)
                        self.textCons.insert(END,message+"\n\n")
                        self.textCons.config(state=DISABLED)
                        self.textCons.see(END)
                except:
                    # an error will be printed on the command line or console if there's an error
                    print("An error occurred!")
                    client.close()
                    break

        # function to send messages
        def sendMessage(self):
            self.textCons.config(state = DISABLED)
            while True:
                message = (f"{self.name}: {self.msg}")
                client.send(message.encode(FORMAT))
                break
        
    
    # create a GUI class object
    g = GUI()



def start_file_transfer():
    file_transfer_window = tk.Toplevel(root)
    file_transfer_window.title("File Transfer Page")
    file_transfer_window.geometry("400x300")
    file_transfer_window.configure(bg='#ff6666')  # Light red background
    chat_windows['file_transfer'] = file_transfer_window

def stop_file_transfer():
    file_transfer_window = chat_windows.get('file_transfer')
    if file_transfer_window:
        file_transfer_window.destroy()

def start_video_conferencing():
    video_conferencing_window = tk.Toplevel(root)
    video_conferencing_window.title("Video Conferencing Page")
    video_conferencing_window.geometry("400x300")
    video_conferencing_window.configure(bg='#6666ff')  # Light blue background
    chat_windows['video_conferencing'] = video_conferencing_window

def stop_video_conferencing():
    video_conferencing_window = chat_windows.get('video_conferencing')
    if video_conferencing_window:
        video_conferencing_window.destroy()

def exit_application():
    client.close()
    root.destroy()

# Create the main window
root = tk.Tk()
root.title("client-client communications")
root.geometry("800x500")  # Wider window for a more organized layout
root.configure(bg='#ffff66')  # Light yellow background

# Heading in the main window
main_heading = tk.Label(root, text="Welcome to SH Client to Client Services", font=('Arial', 20, 'bold'), bg='#ffff66')
main_heading.pack(pady=20)

# Frame for Chat
chat_frame = tk.Frame(root, bg='#ffff66')  # Use the same background color as the main window
chat_frame.pack(pady=10)

chat_heading = tk.Label(chat_frame, text="Chat", font=('TimesNewRoman', 16, 'bold'), bg='#ffff66')
chat_heading.pack(padx=20,pady=10)

chat_start_button = tk.Button(chat_frame, text="Start", command=start_chat, bg='#85ff89')  # Light green button
chat_start_button.pack(side='left', padx=20, pady=5)
#chat_stop_button = tk.Button(chat_frame, text="Stop", command=stop_chat, bg='#ffcccc')  # Light pink button
#chat_stop_button.pack(side='left', pady=5)

# Frame for File Transfer
file_transfer_frame = tk.Frame(root, bg='#ffff66')
file_transfer_frame.pack(pady=10)

file_transfer_heading = tk.Label(file_transfer_frame, text="File Transfer", font=('Arial', 16, 'bold'), bg='#ffff66')
file_transfer_heading.pack(pady=10)

file_transfer_start_button = tk.Button(file_transfer_frame, text="Start", command=start_file_transfer, bg='#ff8c8c')  # Light red button
file_transfer_start_button.pack(side='left', padx=20, pady=5)
file_transfer_stop_button = tk.Button(file_transfer_frame, text="Stop", command=stop_file_transfer, bg='#ffcccc')  # Light pink button
file_transfer_stop_button.pack(side='left', pady=5)

# Frame for Video Conferencing
video_conferencing_frame = tk.Frame(root, bg='#ffff66')
video_conferencing_frame.pack(pady=10)

video_conferencing_heading = tk.Label(video_conferencing_frame, text="Video Conferencing", font=('Arial', 16, 'bold'), bg='#ffff66')
video_conferencing_heading.pack(pady=10)

video_conferencing_start_button = tk.Button(video_conferencing_frame, text="Start", command=start_video_conferencing, bg='#8c8cff')  # Light blue button
video_conferencing_start_button.pack(side='left',padx=40, pady=5)
video_conferencing_stop_button = tk.Button(video_conferencing_frame, text="Stop", command=stop_video_conferencing, bg='#ffcccc')  # Light pink button
video_conferencing_stop_button.pack(side='left', pady=5)

# Exit Button
exit_button = tk.Button(root, text="Exit", command=exit_application, bg='#ff9999')  # Light red button
exit_button.pack(side='bottom', pady=10)

# Dictionary to store references to the opened windows
chat_windows = {}

# Run the Tkinter event loop
root.mainloop()


An error occurred!
